In [1]:
%load_ext micropython_magic
%reload_ext micropython_magic

In [8]:
!mpremote cp -r ../../mp_libs/power :

cp ../../mp_libs/power/__init__.py :../../mp_libs/power/
cp ../../mp_libs/power/bq2562x.py :../../mp_libs/power/
cp ../../mp_libs/power/lc709204f.py :../../mp_libs/power/
cp ../../mp_libs/power/powerfeather.py :../../mp_libs/power/


In [12]:
# %%micropython

import time
from machine import Pin
from mp_libs.power import powerfeather
from mp_libs.power import bq2562x as bq
from mp_libs.power import lc709204f as fg

pf = powerfeather.PowerFeather(batt_type=powerfeather.BatteryType.GENERIC_3V7, batt_cap=1050)

def cb_button(pin: Pin) -> None:
    print("Button Pressed! Toggling charging.")
    if pf._charger.charging_enable:
        pf.batt_charging_enable(False)
    else:
        pf.batt_charging_enable(True)

pf.register_button_irq(cb_button)
term_current = pf._charger.term_current

while True:
    pf.led_toggle()

    supply_voltage = pf.supply_voltage()
    supply_current = pf.supply_current()
    batt_voltage = pf.batt_voltage()
    batt_current = pf.batt_current()
    batt_charge = pf.batt_charge()
    batt_cycles = pf.batt_cycles()
    batt_health = pf.batt_health()
    status = pf.batt_charging_status()

    print(f"[Supply]  Voltage: {supply_voltage} mV\t\tCurrent: {supply_current} mA")
    print(f"[Battery] Voltage: {batt_voltage} mv\t\tCurrent: {batt_current} mA")
    print(f"[Battery] Charge: {batt_charge}%\tCycles: {batt_cycles}\tHealth: {batt_health}%\tStatus: {status}\tTerm: {term_current} mA")
    print(f"Time Left: {pf.batt_time_left()} min")

    time.sleep_ms(300)


[Supply]  Voltage: 5002 mV		Current: 22 mA
[Battery] Voltage: 3933 mv		Current: 0 mA
[Battery] Charge: 71%	Cycles: 0	Health: 100%	Status: Not Charging	Term: 105 mA
2000-01-01 00:14:35.875727 WARNING-PF:Can't yet provide estimate for time left to full/empty.
Time Left: None min
[Supply]  Voltage: 5006 mV		Current: 22 mA
[Battery] Voltage: 3933 mv		Current: 0 mA
[Battery] Charge: 71%	Cycles: 0	Health: 100%	Status: Not Charging	Term: 105 mA
2000-01-01 00:14:35.876435 WARNING-PF:Can't yet provide estimate for time left to full/empty.
Time Left: None min
[Supply]  Voltage: 5002 mV		Current: 22 mA
[Battery] Voltage: 3933 mv		Current: 0 mA
[Battery] Charge: 71%	Cycles: 0	Health: 100%	Status: Not Charging	Term: 105 mA
2000-01-01 00:14:36.877143 WARNING-PF:Can't yet provide estimate for time left to full/empty.
Time Left: None min
[Supply]  Voltage: 5014 mV		Current: 24 mA
[Battery] Voltage: 3933 mv		Current: 0 mA
[Battery] Charge: 71%	Cycles: 0	Health: 100%	Status: Not Charging	Term: 105 mA
20